In [ ]:
import numpy as np
import seaborn as sns
from scipy.integrate import solve_ivp
from SALib.sample import saltelli, sobol, fast_sampler
from SALib.analyze import sobol, fast
from scipy.constants import N_A
from scipy.optimize import fsolve
import math
import matplotlib.pyplot as plt
import SALib

In [ ]:
Target_cell_number = 2e5
well_size = 150e-6
sigma = well_size*N_A/Target_cell_number
A0s = np.geomspace(1e-13, 5e-5, 3)
t_end = 60*60*5
t = np.geomspace(1e-10, t_end, 500)
t_span = [1e-10, t_end]
z0 = [0, 0]
tumour_cell_radius = 8e-6
tumour_cell_surface_area = 4*math.pi*((tumour_cell_radius)**2)

In [ ]:
def A1_steady_state(x, Ainit, k1, koff, k2, rtot):
    k1 = k1/sigma
    Atot = sigma*Ainit
    express = 2*k1*(rtot - x - 2*(k2*x*(rtot-x)/(2*(koff + k2*x))))*(Atot - x - (k2*x*(rtot-x)/(2*(koff + k2*x)))) -koff*x - k2*x*(rtot - x - 2*(k2*x*(rtot-x)/(2*(koff + k2*x)))) +2*koff*(k2*x*(rtot-x)/(2*(koff + k2*x)))

    return express

def A2_steady_state(x, k2, koff, rtot):
    express = k2*x*(rtot-x)/(2*(koff + k2*x))

    return express

def EC50_finder(array, A0s):
    half_max = 0.5*np.max(array) 
    half_max_array = half_max*np.ones_like(array)
    indicies = np.argwhere(np.diff(np.sign(half_max_array-array)))
    return A0s[indicies[0]]

def mono_valent_steady_state(Ainit, k1, koff, rtot):
    k1 = k1/sigma
    Atot = sigma*Ainit
    a = 1
    b = -((koff/k1) + rtot + Atot)
    c = rtot*Atot

    st = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)

    return st

In [ ]:
problem = {
    'num_vars': 4,
    'names': ['rtot', 'kon', 'D', 'koff'],
    'bounds': [[1e3, 1e6],
               [1e4, 1e6],
               [1e-15, 1e-13],
               [1e-6, 1e-3]]
}

vals = SALib.sample.sobol.sample(problem, 8)
Y = np.zeros(len(vals))

In [ ]:
for i, params in enumerate(vals):
    rtot = params[0]
    kon = params[1]
    D = params[2]
    koff = params[3]
    k2 = 4*D/tumour_cell_surface_area

    Ainit_array = np.zeros_like(A0s)

    for j, A0 in enumerate(A0s):

        A1_st = fsolve(A1_steady_state, [0], args=(A0, kon, koff, k2, rtot))
        A2_st = A2_steady_state(A1_st, k2, koff, rtot)
        if (A1_st < 0) or (A2_st < 0) :
            print('negative')
        
        if ((A1_st + 2*A2_st) > rtot):
            print('blown up')

        Ab = A1_st + 2*A2_st

        Ainit_array[j] = Ab
    
    mono_binding = mono_valent_steady_state(A0s, kon, koff, rtot)
    mono_ec50 = EC50_finder(mono_binding, A0s)
    biv_ec50 = EC50_finder(Ainit_array, A0s)
    Y[i] = np.abs(mono_ec50-biv_ec50)

In [ ]:
Si = sobol.analyze(problem, Y, print_to_console=True)

In [ ]:
total, first, second = Si.to_df()
sns.set_context('talk')
from SALib.plotting.bar import plot as barplot

fig, ax1 = plt.subplots(1,1, figsize=(10,6))

ax1 = barplot(total, ax=ax1)

In [ ]:
sns.barplot(data=total, x=total.index, y='ST')

In [ ]:
array = list(first['S1'].values)
array.append(0)

In [ ]:
plt.bar(x =[r'$r^{tot}$', r'$k^{on}$', r'$D$', r'$k^{off}$', 'dummy'],  height=array, color=['purple', 'blue', 'skyblue', 'orange', 'black'])
plt.ylabel('Index Value')
plt.xlabel('Model Parameter')